In [2]:
import os

data_files_list = ["./export/"+f for f in os.listdir("./export") ]
instance_dict = {}
for dir_str in data_files_list:
    with open(dir_str, 'r') as text_file:
        cnt = 0
        instance = ""
        for line in text_file:
            if cnt < 9:
                if cnt == 0:
                    instance = line.split()[0]
                    instance_dict[instance] = []
                cnt += 1
                continue
            split_line = line.split()
            instance_dict[instance].append([int(i) for i in split_line])
        text_file.close()

ng_dict = {}
cnt = -1
with open("ng_outs.csv", 'r') as text_file:
    for line in text_file:
        if cnt < 2:
            cnt += 1
            continue
        raw_line = line.strip()
        split_line_list = raw_line.split(sep=";")
        instance = split_line_list[3]
        if instance not in ng_dict:
            ng_dict[instance] = [[0 for i in range(101)]]
        ng_dict[instance].append([0] + [int(i) for i in split_line_list[5:-1]])
        if len(split_line_list[5:-1]) != 100:
            print("case found for instance "+instance)
    text_file.close()

In [3]:
from math import sqrt
import torch
import torch_geometric as tg
from torch_geometric.data import Data
import networkx as nx
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, SAGEConv, GraphConv, global_add_pool
from torch_geometric.loader import DataLoader

In [4]:
complete_graph_list = []
for i in range(101):
    for j in range(101):
        if i != j:
            complete_graph_list.append([i,j])
edge_index = torch.tensor(complete_graph_list, dtype=torch.long).t().contiguous()
n_edges = len(complete_graph_list)

In [5]:
for instance_name in ng_dict:
    for i in range(101):
        for j in range(101):
            if i == j:
                ng_dict[instance_name][i][j] = 0

In [6]:
data_list = []
for instance_name in ng_dict:
    y = torch.tensor(ng_dict[instance_name], dtype=torch.double)
    x = torch.tensor(instance_dict[instance_name], dtype=torch.double)
    attr = [[i] for i in range(n_edges)]
    loc_dict = {(i[0],j[0]): sqrt((i[1]-j[1])**2 + (i[2]-j[2])**2) for i in instance_dict[instance_name] for j in instance_dict[instance_name]}
    cnt = -1
    for i in range(101):
        for j in range(101):
            if i != j:
                cnt += 1
                attr[cnt].append(loc_dict[i,j])
    attr = torch.tensor(attr, dtype=torch.double)
    pos = []
    for i in instance_dict[instance_name]:
        pos.append([i[1], i[2]])
    pos = torch.tensor(pos, dtype=torch.double)
    data_list.append(Data(x=x, y=y, edge_index=edge_index, pos=pos, edge_attr=attr))

In [7]:
# Just to produce edges as an adjacency matrix
complete_adj_matrix_list = [[0 for i in range(101)] for i in range(101)]
for edge in complete_graph_list:
    i, j = edge
    complete_adj_matrix_list[i][j] = 1

In [8]:

class Instances:
    def __init__(self, data_list):
        self.data_list = data_list
    
    def to_torch_geometric(self, start=0, end=-1, batch_size=1):
        return DataLoader(data_list[start:end], batch_size=batch_size)
    
    def to_conv_nets(self, start=0, end=-1, batch_size=1):
        final_data = []
        nodes = []
        nodes_coor = []
        nodes_timew = []
        x_edges = []
        x_edges_values = []
        y_edges = []
        cnt = 0
        current_batch = 0
        for graph in self.data_list[start:end]:
            if cnt >= batch_size:
                cnt = 0
                current_batch += 1
                nodes = torch.tensor(nodes, dtype=torch.long)
                nodes_coor = torch.tensor(nodes_coor, dtype=torch.float)
                nodes_timew = torch.tensor(nodes_timew, dtype=torch.long)
                x_edges = torch.tensor(x_edges, dtype=torch.long)
                x_edges_values = torch.tensor(x_edges_values, dtype=torch.float)
                y_edges = torch.tensor(y_edges, dtype=torch.long)
                final_data.append((x_edges, x_edges_values, nodes, nodes_coor, nodes_timew, y_edges))
                nodes = []
                nodes_coor = []
                nodes_timew = []
                x_edges = []
                x_edges_values = []
                y_edges = []
            nodes.append([i for i in range(101)]) 
            nodes_coor.append(graph.pos.tolist())
            tw = []
            x_raw = graph.x.tolist()
            for i in range(101):
                tw.append([x_raw[i][4], x_raw[i][5]])
            nodes_timew.append(tw)
            x_edges.append(complete_adj_matrix_list)
            dist_matrix = [[0 for _ in range(101)] for _ in range(101)]
            dist_list = [i for _, i in graph.edge_attr.tolist()]
            pos_dist = 0
            for i in range(101):
                for j in range(101):
                    if i != j:
                        dist_matrix[i][j] = dist_list[pos_dist]
                        pos_dist += 1
            x_edges_values.append(dist_matrix)
            y_edges.append(graph.y.tolist()) #TODO: remove the transpose and also the contiguous when generating y
            cnt += 1
        return final_data


In [9]:
data_source = Instances(data_list)
dataloader = data_source.to_torch_geometric(start=428, end=458, batch_size=1)
data_test = data_source.to_torch_geometric(start=458, end=468, batch_size=1)
# datatorch = data_source.to_conv_nets(start=428, end=1458, batch_size=10)
# torchtest = data_source.to_conv_nets(start=1458, end=488, batch_size=10)

In [10]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        num_features = 7
        dim = 101*101

        self.conv1 = GraphConv(num_features, dim)
        self.conv2 = GraphConv(dim, dim)

        self.lin1 = Linear(dim, dim)

        self.double()

    def forward(self, x, edge_index, batch, edge_weight=None):
        # x, edge_index = data.x, data.edge_index

        # x = self.conv1(x, edge_index)
        # x = F.relu(x)
        # x = F.sigmoid(self.conv2(x, edge_index))
        # x = F.dropout(x, training=self.training)
        # x = self.conv2(x, edge_index)

        x = self.conv1(x, edge_index, edge_weight).relu()
        x = self.conv2(x, edge_index, edge_weight).relu()
        x = global_add_pool(x, batch)
        x = self.lin1(x).relu()
        x = F.dropout(x, p=0.5, training=self.training)
        # x = self.lin2(x)
        x = torch.reshape(x, (101, 101))
        return x

In [11]:
def train(epoch):
    model.train()

    if epoch == 51:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.5 * param_group['lr']

    loss_all = 0
    for data in dataloader:
        data = data.to(device)
        optimizer.zero_grad()
        # print(data.x, data.edge_index, data.batch)
        output = model(data.x, data.edge_index, data.batch)
        # print(output, data.y)
        # print(output)
        loss = F.l1_loss(output, data.y)
        loss.backward()
        loss_all += loss.item() * data.num_graphs
        optimizer.step()
        test_size = 101*100*1 #extract batch size
    return loss_all / test_size

def test(loader):
    model.eval()

    correct = 0
    for data in loader:
        data = data.to(device)
        output = model(data.x, data.edge_index, data.batch)
        pred = output.max(dim=1)[1]
        correct += pred.eq(data.y).sum().item()
        test_size = 101*100*1 #extract batch size
    return correct / test_size

In [12]:
from copy import deepcopy

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model_dict = {}

for epoch in range(1, 11):
    loss = train(epoch)
    train_acc = test(dataloader)
    test_acc = test(data_test)
    model_dict[epoch] = {"model": deepcopy(model), "loss": loss, "trainAcc": train_acc, "testAcc": test_acc}
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, '
          f'Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 001, Loss: 12404.8895, Train Acc: 27.9051, Test Acc: 9.3078
Epoch: 002, Loss: 16.6835, Train Acc: 29.7397, Test Acc: 9.9231
Epoch: 003, Loss: 5.7465, Train Acc: 28.8575, Test Acc: 9.6248
Epoch: 004, Loss: 2.2811, Train Acc: 29.7014, Test Acc: 9.9246
Epoch: 005, Loss: 5.9399, Train Acc: 28.5488, Test Acc: 9.5261
Epoch: 006, Loss: 8.6129, Train Acc: 29.1063, Test Acc: 9.6251
Epoch: 007, Loss: 10.0299, Train Acc: 29.4419, Test Acc: 9.8234
Epoch: 008, Loss: 10.0956, Train Acc: 28.2541, Test Acc: 9.4272
Epoch: 009, Loss: 6.9182, Train Acc: 29.7095, Test Acc: 9.9428
Epoch: 010, Loss: 3.1130, Train Acc: 29.4424, Test Acc: 9.8240


In [14]:
import pickle

In [15]:
with open('torch_geometric.pickle', 'wb') as handle:
    pickle.dump(model_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('filename.pickle', 'rb') as handle:
#     model_dict_restored = pickle.load(handle)

In [16]:
model.eval()
data_show = data_list[430]
pred = model(data_show.x, data_show.edge_index, data_show.batch).tolist()
y = data_show.y
diff = 0
for j in range(len(pred)):
    a_list = [1 if i*100 > 1 else 0 for i in pred[j]]
    diff += sum([(y[j][i].tolist() - a_list[i])**2 for i in range(len(a_list))])
    print([1 if i*10 > 0.1 else 0 for i in pred[j]], sum(a_list))
print(diff)
print(sum([y[j][i].tolist() for i in range(101) for j in range(101)]))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 0
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 